In [1]:
import os
import json
import numpy as np
import pandas as pd
from collections import Counter,defaultdict

In [2]:
with open('data/train.json',encoding='utf-8-sig') as f:
    train_dict = json.load(f)
    
with open('data/song_meta.json',encoding='utf-8-sig') as f:
    song_dict = json.load(f)
    
with open('data/genre_gn_all.json',encoding='utf-8-sig') as f:
    genre_dict = json.load(f)
    
train_df = pd.DataFrame.from_dict(train_dict)
song_df = pd.DataFrame.from_dict(song_dict)

# 데이터 처리

## train_df 전처리

In [3]:
train_df.head(3)

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000


### train_df 필요한 컬럼 추가

In [4]:
# 플레이리스트 곡수 컬럼 추가
train_df['tags_cnt'] = train_df['tags'].map(lambda x : len(x))

# 플레이리스트 태그수 컬럼 추가
train_df['songs_cnt'] = train_df['songs'].map(lambda x : len(x))

### train_df 에 포함된 곡들 (중복포함,중복제거)

In [5]:
from itertools import chain

# 플레이리스트 포함된 노래 중복포함
songs_duplicate = chain.from_iterable(train_df['songs'].tolist())

# 플레이리스트 포함된 노래 중복제거
songs_unique = list(set(songs_duplicate))

### train_df 에 포함된 태그들 (중복포함,중복제거)

In [6]:
# 플레이리스트 포함된 태그 중복포함
tags_duplicate = list(chain.from_iterable(train_df['tags'].tolist()))

# 플레이리스트 포함된 태그 중복제거
tags_unique = list(set(tags_duplicate))

### tag에 새로운 id부여, new_tags_id 컬럼 생성

In [7]:
# { 태그 : 새로운id } 딕셔너리
tag_to_id = dict(zip(tags_unique,range(0,len(tags_unique))))

# { 새로운id : 태그 } 딕셔너리
id_to_tag = dict(zip(range(0,len(tags_unique)),tags_unique))

train_df['new_tags_id'] = train_df['tags'].map(lambda x : [tag_to_id[v] for v in x])

### songs에 새로운 id부여, new_songs_id 컬럼 생성

In [8]:
# { 노래 : 새로운id } 딕셔너리
song_to_id = dict(zip(songs_unique,range(0,len(songs_unique))))

# { 새로운id : 태그 } 딕셔너리
id_to_song = dict(zip(range(0,len(songs_unique)),songs_unique))

train_df['new_songs_id'] = train_df['songs'].map(lambda x : [song_to_id[v] for v in x])

In [9]:
train_df.head(3)

,tags,id,plylst_title,songs,like_cnt,updt_date,tags_cnt,songs_cnt,new_tags_id,new_songs_id
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,1,19,[25187],"[456704, 112732, 333158, 488440, 258853, 12127..."
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000,2,42,"[1886, 7966]","[375894, 587314, 431997, 104605, 338568, 21226..."
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000,2,28,"[815, 15252]","[72132, 240434, 144495, 161861, 307991, 222934..."


In [10]:
train_df = train_df[['id','plylst_title','tags','new_tags_id','songs','new_songs_id','tags_cnt','songs_cnt','like_cnt','updt_date']]
train_df.head(3)

,id,plylst_title,tags,new_tags_id,songs,new_songs_id,tags_cnt,songs_cnt,like_cnt,updt_date
0,61281,여행같은 음악,[락],[25187],"[525514, 129701, 383374, 562083, 297861, 13954...","[456704, 112732, 333158, 488440, 258853, 12127...",1,19,71,2013-12-19 18:36:19.000
1,10532,요즘 너 말야,"[추억, 회상]","[1886, 7966]","[432406, 675945, 497066, 120377, 389529, 24427...","[375894, 587314, 431997, 104605, 338568, 21226...",2,42,1,2014-12-02 16:19:42.000
2,76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[까페, 잔잔한]","[815, 15252]","[83116, 276692, 166267, 186301, 354465, 256598...","[72132, 240434, 144495, 161861, 307991, 222934...",2,28,17,2017-08-28 07:09:34.000


In [11]:
train_df.columns = ['플리_id','플리제목','태그','새태그_id','노래_id','새노래_id','태그수','노래수','좋아요수','갱신일']
train_df.head(3)

,플리_id,플리제목,태그,새태그_id,노래_id,새노래_id,태그수,노래수,좋아요수,갱신일
0,61281,여행같은 음악,[락],[25187],"[525514, 129701, 383374, 562083, 297861, 13954...","[456704, 112732, 333158, 488440, 258853, 12127...",1,19,71,2013-12-19 18:36:19.000
1,10532,요즘 너 말야,"[추억, 회상]","[1886, 7966]","[432406, 675945, 497066, 120377, 389529, 24427...","[375894, 587314, 431997, 104605, 338568, 21226...",2,42,1,2014-12-02 16:19:42.000
2,76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[까페, 잔잔한]","[815, 15252]","[83116, 276692, 166267, 186301, 354465, 256598...","[72132, 240434, 144495, 161861, 307991, 222934...",2,28,17,2017-08-28 07:09:34.000


## 변수 정리

<현재>

- tags_duplicate : 태그 중복 포함 리스트
- tags_unique : 태그 중복 제거 리스트
- songs_duplicate : 노래 중복 포함 리스트
- songs_unique : 노래 중복 제거 리스트
 
- tag_to_id : { 태그 : 새로운id } 딕셔너리
- id_to_tag : { 새로운id : 태그 } 딕셔너리
- song_to_id : { 노래 : 새로운id } 딕셔너리
- id_to_song : { 새로운id : 노래 } 딕셔너리

## song_df 전처리

- 칼럼명 한글화
- 수정날짜 데이터 변환
- 필요한 딕셔너리 생성

In [12]:
song_df.head(3)

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2


In [13]:
# 컬럼명 변환

song_df.columns = ['장르_소분류','수정날짜','앨범명','앨범ID','가수ID','노래명','장르_대분류','가수이름','노래ID']
song_df = song_df[['노래ID','노래명','앨범ID','앨범명','가수ID','가수이름','장르_대분류','장르_소분류','수정날짜']]
song_df.head(3)

,노래ID,노래명,앨범ID,앨범명,가수ID,가수이름,장르_대분류,장르_소분류,수정날짜
0,0,Feelings,2255639,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,[2727],[Various Artists],[GN0900],[GN0901],20140512
1,1,"Bach : Partita No. 4 In D Major, BWV 828 - II....",376431,"Bach : Partitas Nos. 2, 3 & 4",[29966],[Murray Perahia],[GN1600],"[GN1601, GN1606]",20080421
2,2,Solsbury Hill (Remastered 2002),4698747,Hit,[3361],[Peter Gabriel],[GN0900],[GN0901],20180518


In [14]:
songid_singer_dict = dict(zip(song_df['노래ID'].tolist(),song_df['가수이름'].tolist()))
songid_genre_b_dict = dict(zip(song_df['노래ID'].tolist(),song_df['장르_대분류'].tolist()))
songid_genre_d_dict = dict(zip(song_df['노래ID'].tolist(),song_df['장르_소분류'].tolist()))

In [15]:
song_df.sort_values(by='수정날짜',ascending=False).head(3)

,노래ID,노래명,앨범ID,앨범명,가수ID,가수이름,장르_대분류,장르_소분류,수정날짜
448286,448286,여수 바다 (Yeosu sea),10403230,고요한 밤바다 구경하기 [여수 바다],[2737142],[무드홀릭 (Moodholic)],[GN1800],[GN1801],20220113
141185,141185,잔잔한 바람이 불어와 (There is a gentle breeze),10403230,고요한 밤바다 구경하기 [여수 바다],[2737142],[무드홀릭 (Moodholic)],[GN1800],[GN1801],20220113
419219,419219,MDKDVV (Gardons le sourire),10420389,MDKDVV (Gardons le sourire),[2764604],[Bryans],[GN1300],[GN1301],20200423


In [16]:
# {노래id : 노래명}을 가진 딕셔너리
songid_songname_dict = dict(zip(song_df['노래ID'].tolist(),song_df['노래명'].tolist()))
#songid_songname_dict

In [17]:
# {노래id : 가수이름}을 가진 딕셔너리
songid_singername_dict = dict(zip(song_df['노래ID'].tolist(),song_df['가수이름'].tolist()))

In [18]:
# 가장 최신 곡의 날짜
most_recent = '20200423'
most_recent[:-4]

'2020'

In [19]:
# 2020년 노래만 추출
song_19_20_df = song_df[(song_df['수정날짜'].str.startswith('2019')) | (song_df['수정날짜'].str.startswith('2020'))]

In [23]:
song_19_20_df = song_19_20_df.reset_index(drop=True)

In [24]:
# 2020년 노래 df의 수정날짜 데이터 형식을 datetime으로 변경
song_19_20_df.loc[:,('수정날짜')] = pd.to_datetime(song_19_20_df['수정날짜'])

# 마찬가지 플레이리스트 갱신일도 날짜 데이터 형식으로
train_df.loc[:,('갱신일')] = pd.to_datetime(train_df['갱신일'])

In [27]:
from datetime import datetime

# 날짜로 검색

# 첫날
date1 = datetime(2020,1,1)

# 마지막 날
date2 = datetime(2020,4,23)

# 상위 몇개

topn = 30

def popular_song_artist_list(date1,date2,topn):
    
    # 특정 날짜들 사이에 수정된 플레이리스트만 가져오기
    specific_date = song_19_20_df[(song_19_20_df['수정날짜'] >= date1) & (song_19_20_df['수정날짜'] <= date2)]

    # 특정 날짜들 사이에 들어있는 노래 id를 list로
    my_song_list = specific_date['노래ID'].tolist()
    
    # 특정 기간에 갱신된 플레이리스트만 가져옴
    specific_date_plysts = train_df[(train_df['갱신일'] >= date1) & (train_df['갱신일'] <= date2)]
    
    # 어뷰징을 회피하기 위해 좋아요 수가 30을 넘은 (좋아요수 75% 위치의 값) 플레이리스트만 사용
    specific_date_plysts = specific_date_plysts.query('좋아요수 > 10')
    
    # 최신곡이 아닌 노래도 포함한 list
    origin_song_list = list(chain.from_iterable(specific_date_plysts['노래_id'].tolist()))

    # 소괄호 안에 있는 연산을 하면 최신곡이 아닌노래가 나오고 그걸 다시 전체에서 빼주면 최신곡만 남음
    filtered_song_set = set(origin_song_list)  - (set(origin_song_list) - set(my_song_list))
    
    # 일단 Couter를 사용해서 모든 노래의 출현 빈도를 구함
    song_counted = Counter(origin_song_list)

    # 그중에 최신 곡의 빈도만 거름
    filtered_song_couted = {k:song_counted[k] for k in filtered_song_set}

    # 빈도로 정렬
    filtered_song_couted  = sorted(filtered_song_couted .items(),key=lambda x : x[1],reverse=True)
    
    # 빈도수 높은 가수 저장할 defaultdict
    artist_ddict = defaultdict(int)
    
    # 빈도수 높은 장르 저장할 defaultdict
    genre_ddict = defaultdict(int)
    
    # 가장많이나온 노래 topn개 선택
    for id,count in filtered_song_couted [0:topn]:
        print(f'{songid_singername_dict[id]} - {songid_songname_dict[id]}, 빈도수 : {count}')
        
        # 가수들에다가 노래 출몰 빈도수 더해주는 식
        for singer_name in songid_singername_dict[id]:
            artist_ddict[singer_name] += count
            
        for genre_big in  songid_genre_b_dict[id]:
            genre_ddict[genre_big] += count
            
    #가수 dict value로 정렬
    artist_tuples = sorted(artist_ddict.items(),key=lambda x : x[1],reverse=True)
    
    #장르 dict value로 정렬
    genre_tuples = sorted(genre_ddict.items(),key=lambda x : x[1],reverse=True)
    
    genre_tuples = [(genre_dict[k],v) for k,v in genre_tuples]
    
    print('\n\n\n')
    print(artist_tuples[:topn])
    
    print('\n\n\n')
    print(genre_tuples[:topn])

print(date1,date2)
print('\n\n\n')
popular_song_artist_list(date1,date2,topn)

2020-01-01 00:00:00 2020-04-23 00:00:00




['송가인'] - 내 마음의 사진, 빈도수 : 104
['지코 (ZICO)'] - 아무노래, 빈도수 : 99
['아이유'] - 마음을 드려요, 빈도수 : 77
['방탄소년단'] - ON, 빈도수 : 71
['pH-1'] - Nerdy Love (Feat. 백예린), 빈도수 : 67
['백예린'] - 다시 난, 여기, 빈도수 : 65
['Justin Bieber'] - Yummy, 빈도수 : 65
['이찬원'] - 진또배기, 빈도수 : 62
['임영웅'] - 바램, 빈도수 : 62
['방탄소년단'] - 00:00 (Zero O’Clock), 빈도수 : 62
['영탁'] - 막걸리 한잔, 빈도수 : 60
['가호 (Gaho)'] - 시작, 빈도수 : 56
['백현 (BAEKHYUN)'] - 너를 사랑하고 있어, 빈도수 : 55
['유산슬', '송가인'] - 이별의 버스 정류장, 빈도수 : 51
['임영웅'] - 어느 60대 노부부이야기, 빈도수 : 51
['Lauv'] - Changes, 빈도수 : 49
['Jeremy Zucker'] - always, i&#39;ll care, 빈도수 : 49
['송가인'] - 花柳春夢(1막2장), 빈도수 : 49
['영탁'] - 찐이야, 빈도수 : 48
['김호중'] - 태클을 걸지마, 빈도수 : 48
['송가인'] - 花柳春夢, 빈도수 : 48
['IZ*ONE (아이즈원)'] - FIESTA, 빈도수 : 48
['정동원'] - 보릿고개, 빈도수 : 47
['방탄소년단'] - Black Swan, 빈도수 : 46
['Anne-Marie'] - Birthday, 빈도수 : 45
['Mae Muller'] - Therapist, 빈도수 : 45
['방탄소년단'] - 친구, 빈도수 : 44
['김필'] - 그때 그 아인, 빈도수 : 43
['방탄소년단'] - Filter, 빈도수 : 43
['ITZY (있지)'] - WANNABE, 빈도수 : 43



